### 10.1 GroupBy Mechanics
- split-apply-combine for describing group operations
- spilt groups based on one or more keys
- can be grouped on rows (axis=0), columns (axis=1)
- each function applied to each group producing a new value
- then combined into a resukt object

In [5]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-0.318677,0.476144
1,a,two,1.644008,0.999897
2,b,one,1.242755,0.108732
3,b,two,-1.031033,-0.987593
4,a,one,-0.182499,1.952582


In [6]:
# compute the mean of data1 column using label key 1
# access data1 and groupby key1

grouped = df['data1'].groupby(df['key1'])
grouped

In [7]:
grouped.mean()

key1
a    0.380944
b    0.105861
Name: data1, dtype: float64

In [9]:
# if pass multiple arrays as list, will get different

means = df['data1'].groupby([df['key1'], df['key2']]).mean()

means

key1  key2
a     one    -0.250588
      two     1.644008
b     one     1.242755
      two    -1.031033
Name: data1, dtype: float64

In [10]:
# the above has a hierarchical index for key1 and key2
# so when unstack, becomes pivot table

means.unstack()

key2,one,two
key1,,
a,-0.250588,1.644008
b,1.242755,-1.031033


In [11]:
# add in series as group keys
# note the series are same length/ rows as the dataframe

states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    1.644008
            2006    1.242755
Ohio        2005   -0.674855
            2006   -0.182499
Name: data1, dtype: float64

In [12]:
# usually the groupinf information is found in the same DataFrame
# can pass the column names as group keys


df.groupby('key1').mean()
# select 1 column to groupby

,data1,data2
key1,,
a,0.380944,1.142875
b,0.105861,-0.439431


In [13]:
df.groupby(['key1', 'key2']).mean()

#2 different columns to groupby

data1     data2
key1 key2                    
a    one  -0.250588  1.214363
     two   1.644008  0.999897
b    one   1.242755  0.108732
     two  -1.031033 -0.987593

In [14]:
# use .size() to find the series containing group sixes

df.groupby(['key1', 'key2']).size()
#note that missing value in group key are excluded from the result

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups
- GroupBy object supports iteration
- Generating a sequence of 2-tuples containing the group name with a chunk of data

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.318677  0.476144
1    a  two  1.644008  0.999897
4    a  one -0.182499  1.952582
b
  key1 key2     data1     data2
2    b  one  1.242755  0.108732
3    b  two -1.031033 -0.987593


In [18]:
# in the case of multiple keys, the first element in the tuple will be a tuple of key values

for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.318677  0.476144
4    a  one -0.182499  1.952582
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.644008  0.999897
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.242755  0.108732
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.031033 -0.987593


In [19]:
# computing a dict of the data pieves as a one-liner

pieces = dict(list(df.groupby('key1')))

pieces['b']

,key1,key2,data1,data2
2,b,one,1.242755,0.108732
3,b,two,-1.031033,-0.987593


In [20]:
# default groupby on axis =0
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
#groupby columns
grouped = df.groupby(df.dtypes, axis= 1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.318677  0.476144
1  1.644008  0.999897
2  1.242755  0.108732
3 -1.031033 -0.987593
4 -0.182499  1.952582
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a columns or subset of columns
while using group by


df.groupby('key1')['data1'] is the same as df['data1'].groupby(df['key1'])
df.groupby('key1')[['data2']] is the same as df['data2']

In [22]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.214363
     two   0.999897
b    one   0.108732
     two  -0.987593

In [24]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     one     1.214363
      two     0.999897
b     one     0.108732
      two    -0.987593
Name: data2, dtype: float64

### Groupibg with Dicts and Series

grouping information may exist in a form other than an array


In [26]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns = ['a', 'b', 'c', 'd', 'e'],
                      index = ['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people

,a,b,c,d,e
Joe,0.796106,-0.348121,-0.407059,-1.465652,0.912411
Steve,0.306366,-0.826095,-0.829795,-0.613116,0.313876
Wes,0.211788,0.071661,-0.084969,-0.240904,-0.559745
Jim,0.558867,-0.581074,1.995187,-0.684667,1.097670
Travis,-0.070575,-0.214505,0.504739,3.667286,0.109167


In [27]:
people.iloc[2:3, [1,2]] = np.nan

people

,a,b,c,d,e
Joe,0.796106,-0.348121,-0.407059,-1.465652,0.912411
Steve,0.306366,-0.826095,-0.829795,-0.613116,0.313876
Wes,0.211788,NaN,NaN,-0.240904,-0.559745
Jim,0.558867,-0.581074,1.995187,-0.684667,1.097670
Travis,-0.070575,-0.214505,0.504739,3.667286,0.109167


In [28]:
# group correspondance for columns and group together the columns

mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd':'blue', 'e':'red', 'f':'orange'}

# construct an array from this dict to groupby and passs the dictionary

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.872711,1.360395
Steve,-1.442911,-0.205853
Wes,-0.240904,-0.347958
Jim,1.310520,1.075463
Travis,4.172025,-0.175913


In [29]:
# same functionality applies to Series
# view as a fixed-size mapping 

map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [31]:
# group by the length of the names

people.groupby(len).sum()

,a,b,c,d,e
3,1.566760,-0.929196,1.588128,-2.391223,1.450336
5,0.306366,-0.826095,-0.829795,-0.613116,0.313876
6,-0.070575,-0.214505,0.504739,3.667286,0.109167


In [32]:
# Mixing functions with arrays, dicts or Series
# to convert them to arrays

key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.211788 -0.348121 -0.407059 -1.465652 -0.559745
  two  0.558867 -0.581074  1.995187 -0.684667  1.097670
5 one  0.306366 -0.826095 -0.829795 -0.613116  0.313876
6 two -0.070575 -0.214505  0.504739  3.667286  0.109167

### Grouping by Index Levels

- hierarchically indexed dataset
- is the ability to affrefate using one of the levels of an axis index

In [33]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names = ['cty' , 'tenor'])

columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [34]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.062588 -0.894164  0.443249  1.024820 -0.386462
1      0.430826 -1.044751 -1.967022 -0.822617 -0.638354
2     -0.570779 -1.235913 -0.697504  0.711907 -0.759953
3     -0.390678  2.023034  0.331092  0.220600  0.254246

In [36]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### 10.2 Data Aggregation

- refer to data transformation that produces scalar values

In [37]:
df

,key1,key2,data1,data2
0,a,one,-0.318677,0.476144
1,a,two,1.644008,0.999897
2,b,one,1.242755,0.108732
3,b,two,-1.031033,-0.987593
4,a,one,-0.182499,1.952582


In [38]:
grouped = df.groupby('key1')

In [39]:
grouped['data1'].quantile(0.9)

key1
a    1.278707
b    1.015376
Name: data1, dtype: float64

In [40]:
# pass any function that aggregates an array to aggregate method

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.962685,1.476438
b,2.273788,1.096326


In [41]:
# can apply describe to the meeting too
# even though they are not aggregations

grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.380944  1.095963 -0.318677 -0.250588 -0.182499  0.730754   
b      2.0  0.105861  1.607811 -1.031033 -0.462586  0.105861  0.674308   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.644008   3.0  1.142875  0.748531  0.476144  0.738021  0.999897   
b     1.242755   2.0 -0.439431  0.775219 -0.987593 -0.713512 -0.439431   

                          
           75%       max  
key1                      
a     1.476240  1.952582  
b    -0.165349  0.108732

### Column-Wise & Multiple Function Application
use the tipping dataset and add a tipping percentage

In [2]:
import pandas as pd
import numpy as np

tips = pd.read_csv('tips.csv')

#Add tip percentage column of total bill to dataframe

tips['tip_pct'] = tips['tip']/ tips['total_bill']

#show the first 6 rows of bill dataframe

tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [3]:
# group the tips data by day and smoker

grouped = tips.groupby(['day', 'smoker'])

grouped

In [6]:
grouped_pct = grouped['tip_pct']

grouped_pct

In [7]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [8]:
# pass a list of functions or function names instead
# get back a DataFrame with column names

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [9]:
# assign name to the name to the function

grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [10]:
functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)
result

C:\Users\daphnekyx\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [11]:
# the above DataFrame has hierarchical columns
# select the column

result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [12]:
# list of tuple with custom names can be passed

ftuples = [('new name', 'mean'), ('2nd name', np.var)]

grouped['tip_pct' , 'total_bill'].agg(ftuples)


C:\Users\daphnekyx\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


tip_pct           total_bill            
             new name  2nd name   new name    2nd name
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [13]:
# apply potentially different function to columns
# pass a dict to agg to contain mapping of column names

grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [14]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes
- remove the hierarcical index using as_index = False to groupby



In [16]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [17]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### 10.3 Apply: General split-apply-combine

General purpose of GroupBy method is apply

In [18]:
# select the tip five tip_pct values by group
# write a function that selects the rows with the largest value

def top( df, n=5, column ='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [19]:
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [20]:
# group by smoker

tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [21]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column ='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [22]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [23]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Suppressing the Group Keys
-resulting object has a hierarchical index
-disable them by passing group_keys= False to groupby

In [24]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [25]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile & Bucket Analysis
- equal-length bucket categorization using cut

In [26]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0     (-0.107, 1.537]
1    (-1.751, -0.107]
2     (-0.107, 1.537]
3     (-0.107, 1.537]
4     (-0.107, 1.537]
5     (-0.107, 1.537]
6    (-1.751, -0.107]
7     (-0.107, 1.537]
8     (-0.107, 1.537]
9     (-0.107, 1.537]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.401, -1.751] < (-1.751, -0.107] < (-0.107, 1.537] < (1.537, 3.181]]

In [28]:
# The categorical object returned by cut can be passed directly to groupby
# comput a set of statistics for data2 column

def get_stats(group):
    return {'min':group.min(), 'max':group.max(),
            'count':group.count(), 'mean':group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.401, -1.751]",-2.171071,3.154048,46.0,-0.054146
"(-1.751, -0.107]",-2.497220,3.177410,395.0,0.127621
"(-0.107, 1.537]",-3.395966,2.830505,490.0,-0.058130
"(1.537, 3.181]",-2.530110,1.901049,69.0,-0.057723


In [30]:
# Return quartile numbers
# to compute equal-size buckets based on sample quartiles, use qcuts

grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.171071,3.154048,100.0,0.087833
1,-2.265332,3.177410,100.0,0.201110
2,-2.497220,2.294881,100.0,-0.009624
3,-2.023344,2.680682,100.0,0.206370
4,-2.134229,2.012633,100.0,-0.003842
5,-2.235206,2.385615,100.0,-0.076390
6,-3.395966,2.830505,100.0,-0.130190
7,-2.772214,2.239790,100.0,-0.073400
8,-2.762360,2.787321,100.0,0.040425


### Example: Filling Missing Values with Group-Specific Values

In [31]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan

s

0         NaN
1    0.501802
2         NaN
3   -0.228527
4         NaN
5   -2.359519
dtype: float64

In [32]:
s.fillna(s.mean())

0   -0.695414
1    0.501802
2   -0.695414
3   -0.228527
4   -0.695414
5   -2.359519
dtype: float64

In [33]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 
          'Oregon', 'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8), index=states)

data

Ohio         -1.166887
New York     -1.059121
Vermont       0.994409
Florida       1.255915
Oregon        2.343193
Nevada        0.450072
California   -0.291536
Idaho         2.220661
dtype: float64

In [34]:
# set some values in the data to be missing:

data [['Vermont', 'Nevada', 'Idaho']] = np.nan

data

Ohio         -1.166887
New York     -1.059121
Vermont            NaN
Florida       1.255915
Oregon        2.343193
Nevada             NaN
California   -0.291536
Idaho              NaN
dtype: float64

In [35]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [36]:
data.groupby(group_key).mean()

East   -0.323365
West    1.025829
dtype: float64

In [37]:
# fill the NA values using the group means

fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -1.166887
New York     -1.059121
Vermont      -0.323365
Florida       1.255915
Oregon        2.343193
Nevada        1.025829
California   -0.291536
Idaho         1.025829
dtype: float64

In [40]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio         -1.166887
New York     -1.059121
Vermont       0.500000
Florida       1.255915
Oregon        2.343193
Nevada       -1.000000
California   -0.291536
Idaho        -1.000000
dtype: float64

### Example: Random Sampling & Permutation
- construct a deck of cards

In [41]:
# Hearts, Spades, Clubs, Diamonds

suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) +['J', 'K', 'Q']
cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)


deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [42]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

QH    10
AH     1
4C     4
9S     9
4S     4
dtype: int64

In [43]:
# wanted 2 random cards from each suit
# suit is the character of each card name, can group based on this use apply

get_suit = lambda card: card[-1] # last letter is suit

deck.groupby(get_suit).apply(draw, n=2)

C  KC     10
   7C      7
D  KD     10
   8D      8
H  KH     10
   6H      6
S  10S    10
   QS     10
dtype: int64

In [44]:
# Alternatively

deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

7C     7
6C     6
9D     9
AD     1
9H     9
JH    10
AS     1
9S     9
dtype: int64

### Example: Group Weighted Averaged and Correlation

In [46]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

df

,category,data,weights
0,a,0.001508,0.351676
1,a,0.956315,0.942053
2,a,0.047696,0.613956
3,a,0.468947,0.023591
4,b,0.285714,0.876015
5,b,0.093016,0.101669
6,b,-0.820918,0.749574
7,b,-1.002990,0.037322


In [47]:
# group weighted average by category

grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

grouped.apply(get_wavg)


category
a    0.487645
b   -0.222731
dtype: float64

In [48]:
# consider a financial dataset originally obtained from Yahoo! Finance
# containing end-of-day prices for a few stocks and S&P 500

close_px = pd.read_csv('stock_px_2.csv', parse_dates=True,
                        index_col = 0 )

close_px.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [51]:
# see the last 4 lines

close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [52]:
# compute a DataFrame made out of yearly correlations of daily returns

spx_corr = lambda x: x.corrwith(x['SPX'])

rets = close_px.pct_change().dropna()
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [53]:
# compute the annucal correlation between Apple & Microsoft

by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-wise Linear Regression

 - use groupby to perform a complez group-wise statistical analysis

In [55]:
import statsmodels.api as sm
def regress( data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

#run a yearly linear regression of AAPL on SPX returns, execute:

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### 10.4 Pivot Tables & Cross-Tabulation

In [56]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [57]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [58]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns = 'smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [59]:
# include partial totals by passing margins=True

tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns ='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [60]:
# to use a different aggregation function, pass it to aggfunc

tips.pivot_table('tip_pct', index=['time', 'smoker'],
                  columns='day', aggfunc=len, margins=True)



day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [61]:
# some combination as empty
# pass a fill_value

tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns = 'day', aggfunc='mean', fill_value=0)



day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulatons: Crosstabs
- special case of pivot table that computes groups frequencies